In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-classification-for-basic-nlp/Spam Email raw text for NLP.csv


In [6]:
import pandas as pd

df = pd.read_csv('/kaggle/input/spam-classification-for-basic-nlp/Spam Email raw text for NLP.csv')

df.head()

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6


In [7]:
df.tail()

,CATEGORY,MESSAGE,FILE_NAME
5791,0,"I'm one of the 30,000 but it's not working ver...",00609.dd49926ce94a1ea328cce9b62825bc97
5792,0,Damien Morton quoted:\n\n>W3C approves HTML 4 ...,00957.e0b56b117f3ec5f85e432a9d2a47801f
5793,0,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\...",01127.841233b48eceb74a825417d8d918abf8
5794,0,"Once upon a time, Manfred wrote :\n\n\n\n> I w...",01178.5c977dff972cd6eef64d4173b90307f0
5795,0,"If you run Pick, and then use the ""New FTOC"" b...",00747.352d424267d36975a7b40b85ffd0885e


In [9]:
df['CATEGORY'].value_counts()

0    3900
1    1896
Name: CATEGORY, dtype: int64

In [18]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [19]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
test_message = "Hey,, GGggGG feet it going? <HTML> <bads> bads 'randoms' badly"

test_message_tokenized = tokenizer.tokenize(test_message)
test_message_tokenized

['Hey',
 'GGggGG',
 'feet',
 'it',
 'going',
 'HTML',
 'bads',
 'bads',
 'randoms',
 'badly']

In [20]:
test_message_lowercased = [t.lower() for t in test_message_tokenized]

In [21]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

test_message_lemmatized_tokens = [lemmatizer.lemmatize(t) for t in test_message_lowercased]
test_message_lemmatized_tokens 

['hey',
 'gggggg',
 'foot',
 'it',
 'going',
 'html',
 'bad',
 'bad',
 'randoms',
 'badly']

In [23]:
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

test_message_useful_tokens = [t for t in test_message_lemmatized_tokens if t not in stopwords]
test_message_useful_tokens

['hey', 'gggggg', 'foot', 'going', 'html', 'bad', 'bad', 'randoms', 'badly']

In [24]:
def message_to_token_list(s):
    tokens = tokenizer.tokenize(s)
    lowercased_tokens = [t.lower() for t in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
    useful_tokens = [t for t in lemmatized_tokens if t not in stopwords]
    return useful_tokens

In [25]:
message_to_token_list(test_message)

['hey', 'gggggg', 'foot', 'going', 'html', 'bad', 'bad', 'randoms', 'badly']

In [27]:
df = df.sample(frac=1, random_state=1)
df = df.reset_index(drop=True)
split_index = int(len(df)*0.8)
train_df, test_df = df[:split_index], df[split_index:]
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df, test_df

(      CATEGORY                                            MESSAGE  \
 0            1  <HTML>\n\n<BODY>\n\n<META HTTP-EQUIV=3D"Conten...   
 1            1  <html>\n\n\n\n\n\n</head>\n\n\n\n<body>\n\n\n\...   
 2            1  <HTML><HEAD><TITLE>New Web Technology</TITLE>\...   
 3            1  \n\n<HTML>\n\n<HEAD>\n\n<TITLE>bizmagoffer</TI...   
 4            0  URL: http://boingboing.net/#85485824\n\nDate: ...   
 ...        ...                                                ...   
 4631         0  Doom 3 will be based on a peer to peer archite...   
 4632         1  \n\n\n\n<HTML>\n\n<HEAD>\n\n<TITLE>Cards In Ad...   
 4633         1  <BODY BGCOLOR="#FFFFFF">\n\n\n\n<DIV ALIGN="ce...   
 4634         0  Yes - great minds think alike. But even withpu...   
 4635         0  On Tue, Aug 06, 2002 at 04:21:22 +0100, David ...   
 
                                    FILE_NAME  
 0     00275.87c74dc27e397ccd3b2b581bbefef515  
 1     00531.f3fffa4504c7009a03dd0d44a4562a84  
 2     01078.5

In [30]:
token_counter = {}
for message in train_df['MESSAGE']:
    message_as_token_list = message_to_token_list(message)
    for token in message_as_token_list:
        if token in token_counter:
            token_counter[token] += 1
        else:
            token_counter[token] = 1
            
len(token_counter)

76915

In [31]:
token_counter

{'html': 4277,
 'body': 1944,
 'meta': 758,
 'http': 15589,
 'equiv': 448,
 '3d': 34353,
 'content': 2864,
 'type': 2830,
 'text': 2638,
 'charset': 904,
 '3diso': 123,
 '8859': 486,
 '1': 8516,
 'doctype': 115,
 'public': 551,
 'w3c': 123,
 'dtd': 123,
 '4': 3169,
 '0': 9313,
 'transitional': 117,
 'en': 571,
 'head': 1325,
 '3dcontent': 71,
 'mshtml': 85,
 '5': 2989,
 '50': 1643,
 '4134': 19,
 '600': 220,
 'name': 3455,
 '3dgenerator': 67,
 'style': 3307,
 'bgcolor': 2866,
 'ffffff': 2465,
 'div': 3119,
 'nbsp': 10203,
 'font': 34416,
 'size': 13242,
 '3d2': 1043,
 'p': 10622,
 'align': 6914,
 '3dcenter': 715,
 'face': 9616,
 'time': 3142,
 'new': 2651,
 'roman': 498,
 '3d5': 168,
 'em': 433,
 'chickw': 1,
 'eed': 3,
 'healing': 10,
 'salve': 9,
 '3d3': 430,
 'natural': 216,
 'way': 1323,
 'healthy': 37,
 'skin': 54,
 'br': 15303,
 'h': 525,
 'ealing': 1,
 'problem': 961,
 'psoriasis': 1,
 'cancer': 57,
 'rash': 5,
 'sore': 7,
 'throat': 19,
 'di': 30,
 'v': 411,
 'cut': 129,
 'nb': 

In [36]:
def keep_token(processed_token, threshold):
    if processed_token not in token_counter:
        return False
    else:
        return token_counter[processed_token] > threshold
    
keep_token('random', 100)

False

In [37]:
features = set()

for token in token_counter:
    if keep_token(token, 10000):
        features.add(token)
        
features

{'3d', 'b', 'br', 'com', 'font', 'http', 'nbsp', 'p', 'size', 'td', 'tr'}

In [38]:
features = list(features)
features

['nbsp', 'p', 'com', 'font', 'td', 'tr', 'http', 'size', 'br', '3d', 'b']

In [39]:
token_to_index_mapping = {t:i for t, i in zip(features, range(len(features)))}
token_to_index_mapping

{'nbsp': 0,
 'p': 1,
 'com': 2,
 'font': 3,
 'td': 4,
 'tr': 5,
 'http': 6,
 'size': 7,
 'br': 8,
 '3d': 9,
 'b': 10}

In [40]:
"""
Bag of words (counts vector) example :

sentence: "3d d <br> .com bad font font com randoms"

->       nbsp  p  com  font  td  tr  http  size  br  3d  b
-> index  0    1   2    3    4   5    6     7    8    9  10
-> count  [0,  0,  2,   2,   0,  0,   0,    0,   1,   1,  0]
"""

'\nBag of words (counts vector) example :\n\nsentence: "3d d <br> .com bad font font com randoms"\n\n->       nbsp  p  com  font  td  tr  http  size  br  3d  b\n-> index  0    1   2    3    4   5    6     7    8    9  10\n-> count  [0,  0,  2,   2,   0,  0,   0,    0,   1,   1,  0]\n'

In [44]:
import numpy as np

def message_to_count_vector(message):
    count_vector = np.zeros(len(features))
    processed_list_of_tokens = message_to_token_list(message)
    
    for token in processed_list_of_tokens:
        if token not in features:
            continue
        index = token_to_index_mapping[token]
        count_vector[index] += 1
    return count_vector

In [45]:
message_to_count_vector("3d d <br> .com bad font font com randoms")

array([0., 0., 2., 2., 0., 0., 0., 0., 1., 1., 0.])

In [51]:
message_to_count_vector(train_df['MESSAGE'].iloc[3])

array([ 7.,  7., 15., 30., 36., 32., 13., 12.,  7.,  0., 14.])

In [52]:
train_df.iloc[3]

CATEGORY                                                     1
MESSAGE      \n\n<HTML>\n\n<HEAD>\n\n<TITLE>bizmagoffer</TI...
FILE_NAME               00627.4e9619c454da17a27d4a66c87583dd49
Name: 3, dtype: object

In [53]:
def df_to_X_y(dff):
    y = dff['CATEGORY'].to_numpy().astype(int)
    message_col = dff['MESSAGE']
    count_vectors = []
    for message in message_col:
        count_vector = message_to_count_vector(message)
        count_vectors.append(count_vector)
    X = np.array(count_vectors).astype(int)
    return X, y

In [55]:
X_train, y_train = df_to_X_y(train_df)

X_test, y_test = df_to_X_y(test_df)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4636, 11), (4636,), (1160, 11), (1160,))

In [56]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

X_train

array([[0.08818342, 0.02439024, 0.        , ..., 0.03325123, 0.01559454,
        0.2254902 ],
       [0.01587302, 0.1097561 , 0.00947867, ..., 0.00369458, 0.21052632,
        0.31372549],
       [0.        , 0.04065041, 0.01421801, ..., 0.01108374, 0.11111111,
        0.2745098 ],
       ...,
       [0.        , 0.00406504, 0.12322275, ..., 0.00615764, 0.        ,
        0.        ],
       [0.        , 0.        , 0.00473934, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression().fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       790
           1       0.99      0.35      0.52       370

    accuracy                           0.79      1160
   macro avg       0.88      0.68      0.69      1160
weighted avg       0.84      0.79      0.76      1160



In [59]:
# Compare logistic regression to Random Forest classifier
from sklearn.ensemble import RandomForestClassifier 

rf = RandomForestClassifier().fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89       790
           1       0.87      0.59      0.70       370

    accuracy                           0.84      1160
   macro avg       0.85      0.77      0.80      1160
weighted avg       0.85      0.84      0.83      1160

